In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=1000)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [624]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","650.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","653.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("stran") >= 30)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

794838
622937


## Kdy se vyplnovalo pole 655_a?

In [186]:
df.filter(~pl.col("655_a").is_null()).group_by("rok").len().sort(by="rok").join(
    df.group_by('rok').len(), on='rok', how='left'
).with_columns((pl.col("len") / pl.col("len_right")).alias('vyplnenost')).tail(50)

rok,len,len_right,vyplnenost
i64,u32,u32,f64
1976,824,3828,0.215256
1977,899,3795,0.236891
1978,942,3835,0.245632
1979,1148,4019,0.285643
1980,1287,4133,0.311396
1981,1298,3938,0.329609
1982,1647,4033,0.408381
1983,2579,4014,0.642501
1984,2699,4218,0.639877


In [625]:
df = df.filter(pl.col("rok").is_between(1989,2024))

In [190]:
df.filter(pl.col("245_a").str.contains("Každý den je nový")).explode("655_a")

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","""autobiografické komiksy""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","""deníky""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","""autobiographical comics""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","""diaries""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Mandžuková, Jarmila,""","""

In [191]:
df.filter(pl.col("245_a").str.contains("Pod dekou"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Thompson, Craig,""","""xx0037934""","[""aut"", ""ill""]","""1975-""",null,null,null,null,"""nkc20051632284""",""" nam a22 a 4500""","""051123s2005 xr a g 0…","""(váz.) :""","[""Kč 590,00""]","[""80-7341-603-4""]",null,null,null,null,null,null,"""1""","""0""","""Pod dekou""","""ilustrovaný román /""","""Craig Thompson ; [z anglického…",null,null,null,null,null,"[""582 s. :""]","[""il. ;""]","[""25 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","[""americké romány"", ""komiksy"", … ""comics""]","[""fd131796"", ""fd131978"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2005,582,"""pevná"""
"""1""","""Thompson, Craig,""","""xx0037934""","[""aut"", ""ill""]","""1975-""",null,null,null,null,"""nkc20183022395""",""" cam a22 i 4500""","""180919s2018 xr a g 6 0…","""(vázáno)""",null,"[""978-80-7595-075-8""]",null,null,null,null,null,null,"""1""","""0""","""Pod dekou""","""ilustrovaný román /""","""Craig Thompson ; z anglického …",null,null,null,null,null,"[""582 stran :""]","[""ilustrace ;""]","[""24 cm""]",null,null,null,"[""7"", ""9""]","[""komiksy"", ""comics""]","[""fd131978"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1""]","[""Podaný, Richard,""]","[""trl""]","[""1962-""]","[""mzk2003183236""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018,582,"""pevná"""


In [192]:
zebricek = df.explode("655_a").group_by("655_a").len().sort(by="len",descending=True)
zebricek.filter(pl.col('len') >= 200)

655_a,len
str,u32
"""příručky""",38629
"""handbooks and manuals""",29285
"""učebnice vysokých škol""",28897
"""publikace pro děti""",24733
"""children's literature""",20941
"""monografie""",19881
"""textbooks (higher)""",18707
"""monographs""",18058
"""populárně-naučné publikace""",15461


In [193]:
zebricek.filter(pl.col("655_a").str.contains("dívč"))

655_a,len
str,u32
"""dívčí romány""",261


In [622]:
df.sample(20)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Klecker, Dalibor,""","""mzk2004221724""","[""aut""]","""1951-2005""",null,null,null,null,"""cpk20041303068""",""" cam a22 a 4500""","""040205s2004 xr af e 0…","""(brož.)""",null,"[""80-7157-745-6""]",null,null,null,null,null,null,"""1""","""0""","""Nové technologické systémy cho…","""monografie /""","""Dalibor Klecker a kol.""",null,null,null,null,null,"[""47 s., [2] l. barev. obr. příl. :""]","[""il. ;""]","[""22 cm""]",null,null,null,"[""7"", ""9""]","[""studie"", ""studies""]","[""fd133597"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2004,47,"""brožovaná"""
"""1""","""Rudiš, Miroslav""",null,"[""aut""]",null,null,null,null,null,"""np9317893""",""" nam a22 a 4500""","""940418s1993 xr a 0…","""(brož.)""",null,"[""80-901181-4-3""]",null,null,null,null,null,null,"""1""","""0""","""Využití stochastických metod v…","""Application of Stochastic Meth…","""Miroslav Rudiš""",null,null,null,null,null,"[""117 s. :""]","[""grafy ;""]","[""21 cm""]",null,null,null,"[""7""]","[""studie""]","[""fd133597""]","[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1993,117,"""brožovaná"""
"""1""","""Pujman, Petr,""","""jk01101701""","[""aut""]","""1929-1989""",null,null,null,null,"""np9307993""",""" nam a22 4500""","""940120s1992 xr e u0…","""(Brož.) :""","[""Kčs 59,00""]",null,null,null,null,null,null,null,"""1""","""0""","""Prevít a zvířátka""",null,"""Petr Pujman ; [doslov] Mirosla…",null,null,null,null,null,"[""173 s. ;""]",null,"[""21 cm""]",null,null,null,"[""7""]","[""české romány""]","[""fd133974""]","[""czenas""]",null,null,null,null,"[""1""]","[""Petříček, Miroslav,""]","[""aft""]","[""1951-""]","[""jn99240000872""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1992,173,"""brožovaná"""
"""1""","""Mates, Pavel,""","""jk01080708""","[""aut""]","""1947-""",null,null,null,null,"""nkc20091995230""",""" nam a22 a 4500""","""091130s2009 xr e p 0…","""(váz.)""",null,"[""978-80-7408-019-7""]",null,null,null,null,null,null,"""1""","""0""","""Základy správního práva""","""obecná část /""","""Pavel Mates, Marek Starý""",null,null,null,null,null,"[""142 s. ;""]",null,"[""26 cm""]",null,null,null,"[""7"", ""9""]","[""učebnice vysokých škol"", ""textbooks (higher)""]","[""fd133772"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1""]","[""Starý, Marek,""]","[""aut""]","[""1974-""]","[""jo2003170260""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2009,142,"""pevná"""
"""1""","""Revajová, Toňa,""","""ola2002153113""","[""aut""]","""1949-""",null,null,null,null,"""nkc20162836021""",""" cam a22 i 4500""","""160908s2016 xr a c 0…","""(vázáno)""",null,"[

In [614]:
vyvoj = df.filter(pl.col("rok").is_between(2000,2003)).explode("655_a").group_by("655_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("655_a").group_by("655_a").len(), on="655_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")

In [618]:
vyvoj.filter(pl.col("celkem") >= 100).head(150)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""women's novels""",1,"""women's novels""",616,0.001623,617
"""akční a dobrodružné komiksy""",1,"""akční a dobrodružné komiksy""",298,0.003356,299
"""action and adventure comics""",1,"""action and adventure comics""",294,0.003401,295
"""fantasy komiksy""",1,"""fantasy komiksy""",288,0.003472,289
"""fantasy comics""",1,"""fantasy comics""",288,0.003472,289
"""superhrdinské komiksy""",1,"""superhrdinské komiksy""",232,0.00431,233
"""superhero comics""",1,"""superhero comics""",230,0.004348,231
"""romány pro ženy""",3,"""romány pro ženy""",616,0.00487,619
"""autorské knihy""",1,"""autorské knihy""",181,0.005525,182


In [620]:
vyvoj.filter(pl.col("celkem") >= 100).tail(150)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""Irish fiction (English)""",37,"""Irish fiction (English)""",81,0.45679,118
"""komentovaná vydání""",117,"""komentovaná vydání""",254,0.46063,371
"""commentaries""",99,"""commentaries""",214,0.462617,313
"""exhibition catalogs""",152,"""exhibition catalogs""",328,0.463415,480
"""Czech fiction""",1191,"""Czech fiction""",2558,0.465598,3749
"""fiction in English""",73,"""fiction in English""",150,0.486667,223
"""adventure novels""",113,"""adventure novels""",229,0.49345,342
"""katalogy""",93,"""katalogy""",186,0.5,279
"""německé romány""",254,"""německé romány""",488,0.520492,742


In [640]:
df.select(pl.col("653_a")).drop_nulls().sample(50)

653_a
list[str]
"[""kulturistika"", ""racionální výživa"", ""trénink""]"
"[""Vyhnanci"", ""irská divadelní hra""]"
"[""transakční aktivismus""]"
"[""gender mainstreaming""]"
"[""celotělová body masáž"", ""body masáže""]"
"[""integrální andragogika""]"
"[""čas (kategorie estetická)"", ""česká hudba""]"
"[""archetypová psychoterapie""]"
"[""česká fotografie""]"


In [656]:
vyvoj2 = df.filter(pl.col("rok").is_between(2000,2003)).explode("653_a").group_by("653_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("653_a").group_by("653_a").len(), on="653_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).fill_null(0).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")
vyvoj2.filter(pl.col("celkem") > 5)

653_a,len,653_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
null,0,"""jazyková úroveň A0""",6,0.0,6
null,0,"""jazyková úroveň C1""",10,0.0,10
null,0,"""jazyková úroveň A2""",45,0.0,45
null,0,"""jazyková úroveň B1""",33,0.0,33
null,0,"""jazyková úroveň A1""",44,0.0,44
null,0,"""literatura new adult""",58,0.0,58
null,0,null,39681,0.0,39681
null,0,"""jazyková úroveň B2""",21,0.0,21
null,37832,null,0,0.0,37832


In [660]:
vyvoj3 = df.filter(pl.col("rok").is_between(2000,2003)).explode("655_a").group_by("655_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("655_a").group_by("655_a").len(), on="655_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).fill_null(0).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")
vyvoj3.filter(pl.col("celkem") > 100)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
null,0,"""literatura young adult""",390,0.0,390
null,0,"""certifikované metodiky""",174,0.0,174
null,0,"""comics adaptations""",105,0.0,105
null,0,"""komiksové adaptace""",105,0.0,105
null,0,"""manga""",343,0.0,343
null,0,"""girls' novels""",179,0.0,179
null,0,"""certified methodology""",173,0.0,173
null,0,"""young adult literature""",389,0.0,389
"""Adventure stories""",419,null,0,0.0,419


In [485]:
def podil_zanru(zanr):
    return df.explode("655_a").filter(
        pl.col("655_a").str.contains("(?i)" + zanr)
    ).group_by(
        "rok"
    ).len(
    ).join(df.drop_nulls(subset=["655_a"]).group_by("rok").len(), on="rok", how="right").with_columns((pl.col("len") / pl.col("len_right")).alias("podil")).sort(by="rok").rename({'len':'pocet'})

In [198]:
df.explode("655_a").filter(pl.col('655_a') == "erotické romány").group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""Twisted""",6
"""Jezdci apokalypsy""",5
"""Hříšné odstíny vášně""",4
"""This man trilogie""",4
"""Královská sága plná sexu""",4
"""Barvy lásky""",4
"""Vlci zvěrokruhu""",4
"""Hot Jocks""",3
"""Julietta, čili, Slasti neřesti""",3


In [199]:
podil_zanru("komiks")

rok,pocet,len_right,podil
i64,u32,u32,f64
1989,4,3688,0.001085
1990,8,3905,0.002049
1991,8,4096,0.001953
1992,9,4924,0.001828
1993,5,6408,0.00078
1994,12,6650,0.001805
1995,9,7560,0.00119
1996,6,7861,0.000763
1997,13,8379,0.001551


In [200]:
podil_zanru("pohádky")

rok,pocet,len_right,podil
i64,u32,u32,f64
1989,9,3688,0.00244
1990,14,3905,0.003585
1991,19,4096,0.004639
1992,56,4924,0.011373
1993,70,6408,0.010924
1994,54,6650,0.00812
1995,78,7560,0.010317
1996,59,7861,0.007505
1997,97,8379,0.011577


In [201]:
podil_zanru("erotick")

rok,pocet,len_right,podil
i64,u32,u32,f64
1990,11,3905,0.002817
1991,10,4096,0.002441
1992,5,4924,0.001015
1993,6,6408,0.000936
1994,10,6650,0.001504
1996,10,7861,0.001272
1997,14,8379,0.001671
1998,4,8424,0.000475
1999,3,8818,0.00034


In [202]:
podil_zanru("dívčí romány")

rok,pocet,len_right,podil
i64,u32,u32,f64
1989,3,3688,0.000813
1990,1,3905,0.000256
1991,7,4096,0.001709
1992,3,4924,0.000609
1993,1,6408,0.000156
1995,1,7560,0.000132
1996,1,7861,0.000127
1997,1,8379,0.000119
2000,3,9365,0.00032


In [203]:
podil_zanru("populárně-naučné publikace")

rok,pocet,len_right,podil
i64,u32,u32,f64
1989,45,3688,0.012202
1990,52,3905,0.013316
1991,55,4096,0.013428
1992,72,4924,0.014622
1993,130,6408,0.020287
1994,144,6650,0.021654
1995,217,7560,0.028704
1996,184,7861,0.023407
1997,226,8379,0.026972


In [477]:
def grafik(z, funkce=podil_zanru):
    return alt.Chart(alt_friendly(funkce(z))).mark_line().encode(
        alt.X("rok:T",axis=alt.Axis(title=None)),
        alt.Y('podil:Q',axis=alt.Axis(
            labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6',title=None
        ))
    ).configure_view(stroke='transparent')

In [479]:
grafik('erotické')

alt.Chart(...)

In [612]:
grafik('CD-ROM')

alt.Chart(...)

In [441]:
grafik("dvojjazyčná vydání")

alt.Chart(...)

In [208]:
grafik("fantasy romány"	)

alt.Chart(...)

In [209]:
grafik("učebnice základních škol")

alt.Chart(...)

In [210]:
grafik("aforismy")

alt.Chart(...)

In [211]:
grafik("rusk")

alt.Chart(...)

In [487]:
grafik("young adult")

alt.Chart(...)

In [213]:
grafik("příručky")

alt.Chart(...)

In [214]:
grafik("dívčí romány")

alt.Chart(...)

In [215]:
grafik("komiks")

alt.Chart(...)

In [216]:
grafik("detektiv")

alt.Chart(...)

In [217]:
grafik("rozhovory")

alt.Chart(...)

In [218]:
grafik("deníky")

alt.Chart(...)

In [219]:
grafik("autobiogr")

alt.Chart(...)

In [220]:
grafik("encyclopedias")

alt.Chart(...)

In [221]:
grafik("učebnice vysokých škol")

alt.Chart(...)

In [222]:
grafik("kuchař")

alt.Chart(...)

## Kuchařky

In [224]:
kucharky = df.explode("655_a").filter(
        pl.col("655_a").str.contains("kuchař")
    )

In [401]:
kucharky.sample(10)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Novotný, Petr,""","""jn20001031017""","[""aut""]","""1947 srpen 6.-""",null,null,null,null,"""cpk20010886781""",""" cam a22 a 4500""","""000908s2000 xr a e 0…","""(váz.)""",null,"[""80-7181-425-3""]",null,null,null,null,null,null,"""1""","""0""","""Ham a mňam""","""nejlepší recepty z Dobrýho kaf…","""Petr Novotný""",null,null,null,null,null,"[""127 s. :""]","[""barev. il. ;""]","[""24 cm""]",null,null,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2000,127,"""pevná"""
"""1""","""Vlachová, Libuše""","""jk01150037""","[""aut""]",null,null,null,null,null,"""cpk20021092595""",""" nam a22 a 4500""","""020524s2002 xr af e f 0…","""(brož.) :""","[""Kč 80,00""]","[""80-86081-83-4""]",null,null,null,null,null,null,"""1""","""0""","""Pikantní jídla, aneb, Co mužům…",null,"""Libuše Vlachová""",null,null,null,null,null,"[""90 s., [12] s. obr. příl. :""]","[""barev. il. ;""]","[""21 cm""]",null,null,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2002,90,"""brožovaná"""
"""1""","""Filipová, Eva,""","""xx0017475""","[""aut""]","""1957-""",null,null,null,null,"""nkc20102027503""",""" cam a22 a 4500""","""091126s2009 xr a e f 0…","""(váz.)""",null,"[""978-80-904436-0-0""]",null,null,null,null,null,null,"""1""","""0""","""Znovu objevené recepty""",null,"""Eva Filipová ; [foto Iveta Kop…",null,null,null,null,null,"[""197 s. :""]","[""barev. il. ;""]","[""27 cm +""]","[""Pexeso pro malé i velké labužníky ([4] s.)""]",null,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1""]","[""Kopicová, Iveta""]","[""pht""]",null,"[""xx0088504""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2009,197,"""pevná"""
"""1""","""Oliver, Jamie,""","""xx0004981""","[""aut"", ""pht""]","""1975-""",null,null,null,null,"""nkc20183059562""",""" cam a22 i 4500""","""181115s2018 xr ac e f 0…","""(vázáno)""",null,"[""978-80-907344-0-1""]","[""978-80-905003-9-6""]",null,null,null,null,null,"""1""","""0""","""Jamie Oliver vaří po italsku""",null,"""z anglického originálu Jamie c…",null,null,null,null,null,"[""405 stran :""]","[""ilustrace (převážně barevné), portréty ;""]","[""26 cm""]",null,null,null,"[""7""]","""kuchařské recepty""","[""fd132687""]","[""czenas""]",null,null,null,null,"[""1""]","[""Rákosníková, Jitka""]","[""trl""]",null,"[""pna2006337954""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018,405,"""pevná"""
"""1""","""Schmedes, Christa""","""mzk2004246563""","[""au

In [497]:
def kuchyne(slovo):
    return kucharky.filter(
        pl.col("245_a").str.contains("(?i)" + slovo) | pl.col("245_a").str.contains("(?i)" + slovo)
    ).group_by(
        "rok"
    ).len(
    ).join(kucharky.group_by("rok").len(), on="rok", how="right").with_columns((pl.col("len") / pl.col("len_right")).alias("podil")).sort(by="rok").rename({'len':'pocet'}).fill_null(0)

In [499]:
kuchyne('barbe')

pocet,rok,len_right,podil
u32,i64,u32,f64
0,1989,23,0.0
0,1990,32,0.0
0,1991,42,0.0
0,1992,61,0.0
0,1993,89,0.0
0,1994,96,0.0
0,1995,110,0.0
0,1996,103,0.0
0,1997,118,0.0


In [501]:
grafik('barbe', funkce=kuchyne)

alt.Chart(...)

In [503]:
grafik('vegan', funkce=kuchyne)

alt.Chart(...)

In [389]:
grafik('sous ', funkce=kuchyne)

alt.Chart(...)

In [505]:
grafik('svačin', funkce=kuchyne)

alt.Chart(...)

In [509]:
grafik('pečiv', funkce=kuchyne)

alt.Chart(...)

In [511]:
grafik('chleb', funkce=kuchyne)

alt.Chart(...)

In [513]:
grafik('peče', funkce=kuchyne)

alt.Chart(...)

In [515]:
grafik('diabet', funkce=kuchyne)

alt.Chart(...)

In [517]:
grafik('gril', funkce=kuchyne)

alt.Chart(...)

In [521]:
grafik('cukrov', funkce=kuchyne)

alt.Chart(...)

In [523]:
grafik('zdrav', funkce=kuchyne)

alt.Chart(...)

In [525]:
grafik('bez ', funkce=kuchyne)

alt.Chart(...)

In [421]:
kucharky.filter(pl.col("245_a").str.contains("pro")).with_columns(pl.col("245_a").map_elements(lambda x: x.split("pro")[1].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""diabetiky""",33
"""děti""",16
"""cukrářskou výrobu""",14
"""každou příležitost""",12
"""každého""",10
"""zdraví""",9
"""každý den""",9
"""moderní ženu""",9
"""labužníky""",9


In [423]:
kucharky.filter(pl.col("245_a").str.contains(" s ")).with_columns(pl.col("245_a").map_elements(lambda x: x.split(" s ")[1].split(" ")[0].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""láskou""",17
"""potěšením""",6
"""Měsícem""",6
"""Ellou""",5
"""rozumem""",5
"""fantazií""",4
"""konopím""",4
"""dětmi""",4
"""Habadějem""",4


In [323]:
kucharky.filter(pl.col("245_a").str.contains(" bez ")).with_columns(pl.col("245_a").map_elements(lambda x: x.split(" bez ")[1].split(" ")[0].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""lepku""",25
"""vážení""",15
"""cholesterolu""",8
"""cukru""",7
"""mléka""",6
"""pšenice""",4
"""hladovění""",3
"""chemie""",3
"""soli""",2


In [427]:
grafik(' s ', funkce=kuchyne)

alt.Chart(...)

In [429]:
grafik(' bez ', funkce=kuchyne)

alt.Chart(...)

In [527]:
grafik('frit', funkce=kuchyne)

alt.Chart(...)

In [549]:
grafik('indi', funkce=kuchyne)

alt.Chart(...)

In [547]:
grafik('japon', funkce=kuchyne)

alt.Chart(...)

In [545]:
grafik('vietn', funkce=kuchyne)

alt.Chart(...)

In [533]:
grafik('pán[ev]', funkce=kuchyne)

alt.Chart(...)

In [609]:
grafik('mikrov', funkce=kuchyne)

alt.Chart(...)

In [607]:
grafik('gril', funkce=kuchyne)

alt.Chart(...)

In [605]:
grafik('sex', funkce=kuchyne)

alt.Chart(...)

In [601]:
grafik('babi', funkce=kuchyne)

alt.Chart(...)

In [483]:
grafik('pomaz', funkce=kuchyne)

alt.Chart(...)

In [481]:
grafik('cukr', funkce=kuchyne)

alt.Chart(...)

In [553]:
grafik('hrní', funkce=kuchyne)

alt.Chart(...)

In [557]:
grafik('hubn', funkce=kuchyne)

alt.Chart(...)

In [561]:
grafik('omáč', funkce=kuchyne)

alt.Chart(...)

In [559]:
grafik('pol[íé]v', funkce=kuchyne)

alt.Chart(...)

In [551]:
grafik('pomaz', funkce=kuchyne)

alt.Chart(...)

In [567]:
grafik('keto', funkce=kuchyne)

alt.Chart(...)

In [565]:
grafik('paleo', funkce=kuchyne)

alt.Chart(...)

In [575]:
grafik('houb', funkce=kuchyne)

alt.Chart(...)

In [597]:
grafik('(kvas|kvaš|ferment)', funkce=kuchyne)

alt.Chart(...)

In [599]:
grafik('makrobiot', funkce=kuchyne)

alt.Chart(...)

In [593]:
grafik('bílkov', funkce=kuchyne)

alt.Chart(...)

In [595]:
kucharky.select(pl.col("245_a")).to_series().to_list()

['Cukrařinka',
 'Sláma v botách',
 'Česká kuchařka podle Miloše Štěpničky',
 'Klobásy, salámy, paštiky',
 'Helenčino pečení',
 'Cat & cook',
 'Kuchařka pro mého muže',
 'Slezská kuchyně pro den všední i sváteční',
 'Těstoviny',
 'Vzorné řeznictví a uzenářství',
 'Luštěniny',
 'Bio&dítě',
 'Květinový rok',
 'Snídaně, přesnídávky a svačiny pro školáky',
 'Cesta kolem světa v 2krát osmdesáti receptech',
 '500 veganských receptů',
 'Saláty',
 '1001 receptur cocktailů',
 'Špagety, makaróny, lasagne',
 'Vaříme a pečeme bez vážení',
 'Zdravá požitkářka',
 'Dobroty z čokolády',
 'Koláče, dorty a bábovky',
 'Pečeme bez lepku',
 'Vaříme s Habadějem 1998',
 'Paleta nápadů pro malé výtvarníky',
 'Atlas polévek',
 'Džemy a zavařeniny',
 'Strava pro běžce',
 'Kotlety, řízky, špízy--',
 'Zeleninová jídla v mikrovlnné troubě',
 'Dvanáct chutí roku',
 'Kuchyně staré Šumavy',
 'Kuchařka ze spíže',
 'Cucina vegetariana',
 'Bezezbytková dieta při onemocnění střev',
 'Někdo to rád pálivé',
 'Netradiční veg